Goal: same staged-unfreeze/AMP/EMA pipeline, now reading ROI manifests, optional topK subset, mixup, focal loss, and weighted sampler. (You already have Cell A/B/C; here’s what to add/change.)

In [ ]:
### Cell A (Config): set the manifest you want:
from pathlib import Path, json
root = Path("..").resolve()

CONFIG = {
  "manifest": "data/wlasl_preprocessed/manifest_nslt2000_roi_top300.csv",  # or ..._roi.csv for full set
  "clip_len": 32, "frame_stride": 2, "batch_size": 16, "num_workers": 8,
  "use_weighted_sampler": True,
  "normalize": "kinetics",
  "epochs": 50, "warmup_epochs": 3,
  "lr_head_stage1": 1e-3, "lr_back_stage1": 0.0,
  "lr_head_stage2": 1e-4, "lr_back_stage2": 3e-5,
  "weight_decay": 1e-5, "cosine_eta_min": 1e-5,
  "amp": True, "compile": True, "freeze_backbone": True,
  "unfreeze_at_epoch": 5, "unfreeze_scope": "layer4",
  "ema": True, "ema_decay": 0.999,
  "mixup_alpha": 0.2, "cutmix_alpha": 0.0,
  "grad_clip_norm": 1.0,
  "seed": 42,
}


In [ ]:
#Cell B (Dataset/Loader): same as you used, plus random temporal start offset for training (you already have the patch idea).
# losses.py (or inline)
import torch, torch.nn as nn
class FocalLoss(nn.Module):
    def __init__(self, gamma=1.5, weight=None, reduction="mean"):
        super().__init__()
        self.gamma = gamma; self.weight = weight; self.reduction = reduction
        self.ce = nn.CrossEntropyLoss(weight=weight, reduction="none")
    def forward(self, logits, target):
        ce = self.ce(logits, target)
        pt = torch.exp(-ce)
        fl = ((1-pt)**self.gamma) * ce
        return fl.mean() if self.reduction=="mean" else fl.sum()

# Load optional weights
import json, os
weights_path = root/"runs"/"class_weights_top300.json"
if weights_path.exists():
    from collections import OrderedDict
    w = json.load(open(weights_path))
    max_label = max(map(int, w.keys()))
    weight_vec = torch.tensor([w.get(str(i), 1.0) for i in range(max_label+1)], dtype=torch.float)
else:
    weight_vec = None

use_focal = True
criterion = FocalLoss(gamma=1.5, weight=weight_vec.to(device) if weight_vec is not None else None) \
            if use_focal else nn.CrossEntropyLoss(weight=weight_vec.to(device) if weight_vec is not None else None)
